In [1]:
%pip install seaborn
pip install sklearn.dataset
pip install scikit-learn

     -------------------------------------- 293.3/293.3 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 3.3 MB/s eta 0:00:00
     -------------------------------------- 163.0/163.0 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 318.0 kB/s eta 0:00:00
     -------------------------------------- 55.4/55.4 kB 960.7 kB/s eta 0:00:00
     ---------------------------------------- 98.3/98.3 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import nltk 
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import pickle

In [53]:
df_data = pd.read_csv("C:/Users/abcd/Downloads/FINAL_PM_Modi_Speech_Cleaned_string.csv")

In [54]:
df_data

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,['governor andhra pradesh shri e l narasimhan ...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,['chief minister gujarat shri vijay rupani ji ...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,['delighted gift city inaugurate india first i...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,['congratulate finance minister arun jaitley j...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,['sri pejavar math respected sri vishwesh tirt...,sri pejavar math respected sri vishwesh tirth ...
...,...,...,...,...,...,...
1177,NMS1178,2023-04-09,My colleagues in the Union Cabinet Shri Bhupen...,English rendering of PM’s address at the inaug...,['colleague union cabinet shri bhupender yadav...,colleague union cabinet shri bhupender yadav j...
1178,NMS1179,2023-04-12,Namaskar! \n\nThe campaign to provide governme...,English rendering of PM’s address at training ...,"['namaskar', 'campaign provide government job ...",namaskar campaign provide government job youth...
1179,NMS1180,2023-04-12,"Namaskar, Governor of Rajasthan Shri Kalraj Mi...",English rendering of PM’s address during flagg...,['namaskar governor rajasthan shri kalraj mish...,namaskar governor rajasthan shri kalraj mishra...
1180,NMS1181,2023-04-14,"Governor of Assam Shri Gulab Chand Kataria ji,...",English rendering of PM’s address at inaugurat...,['governor assam shri gulab chand kataria ji c...,governor assam shri gulab chand kataria ji chi...


In [3]:

def model(corpus):
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]
    model = Word2Vec(sentences=corpus_tokenized, vector_size=300, window=5, min_count=5, workers=4)
    return model 


In [4]:

def predict(model, speech):
    vectors = []

    for sentence in speech:
        sent_vec = []
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                sent_vec.append(word_vector)
        vectors.append(sent_vec)

    return vectors 


In [5]:

def count_unique_words(j):
    m = j.replace('[','').replace(']','').replace("'","").split(', ')
    l = []
    for k in m:
        l += k.split(" ")
    length = len(l)
    return length 


In [6]:
corpus_Modi = []
for i in df_data['Cleaned_Speech']:
    corpus_Modi += i.replace('[','').replace(']','').replace("'","").split(', ')


In [7]:
len(corpus_Modi)

249435

In [8]:
model_modi = model(corpus_Modi)

In [9]:
Word2vec_model_NM = open("Word2vec_model_NM.pickle", "wb")
pickle.dump(model_modi, Word2vec_model_NM)
Word2vec_model_NM.close()

In [10]:
speech_modi = []
for i in df_data['Cleaned_Speech']:
    speech_modi.append(i.replace('[','').replace(']','').replace("'","").split(', '))


In [22]:
len(speech_modi)

1182

In [20]:
count = 0
for sp in speech_modi:
    for sent in sp:
        if len(sent)==0:
            count +=1


In [21]:
count

0

In [19]:
speech_modi = [[sent for sent in sp if len(sent) > 0] for sp in speech_modi]

In [23]:
speech_modi_vector = []
for sp in speech_modi:
    speech_modi_vector.append(predict(model_modi, sp)) 

In [3]:
len(corpus_Modi)

249435

In [4]:
len(speech_modi_vector[1][163])

120

In [24]:
import pandas as pd
import numpy as np

# initialize an empty list to store the row dictionaries
rows = []

# loop over all speeches, sentences, and words
for speech_number, speech in enumerate(speech_modi_vector):
    for sentence_number, sentence in enumerate(speech):
        # initialize a vector of zeros with 300 dimensions
        sentence_vector = np.zeros(300)
        for word in sentence:
            # add the ith element of the word vector to the corresponding dimension of the sentence vector
            sentence_vector += np.array(word)
        # divide the sentence vector by the number of words to get the average
        sentence_vector /= len(sentence)
        # create a dictionary with the speech number, sentence number, and sentence vector
        row_dict = {"speech_number": speech_number, "sentence_number": sentence_number}
        for i, dim in enumerate(sentence_vector):
            row_dict["dim_" + str(i)] = dim
        # add the dictionary to the list of rows
        rows.append(row_dict)

# create a dataframe from the list of rows
df = pd.DataFrame(rows)


In [25]:
df

,speech_number,sentence_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,0,0,-0.227047,-0.109136,-0.153907,0.103524,-0.476021,0.173697,0.060143,-0.072933,...,0.124696,0.615699,-0.128328,-0.350778,0.368327,0.331424,0.071554,-0.054925,0.089649,0.203533
1,0,1,-0.141709,-0.120520,-0.131730,0.035800,-0.430118,0.115733,0.056891,-0.064264,...,0.106243,0.573977,-0.125105,-0.282681,0.275835,0.375765,0.100995,-0.053994,0.034688,0.182475
2,0,2,-0.181088,-0.068253,-0.079953,0.055245,-0.405421,0.088961,0.113497,-0.052174,...,0.114114,0.501927,-0.110752,-0.292538,0.253770,0.346458,0.083194,-0.078981,0.051047,0.201261
3,0,3,-0.211178,-0.114367,-0.171721,0.080453,-0.391387,0.183108,0.049930,-0.090253,...,0.037537,0.490561,-0.106524,-0.268397,0.297861,0.317943,0.072508,-0.052420,0.022274,0.166357
4,0,4,0.034150,-0.245213,-0.104977,-0.142941,-0.405690,-0.077476,-0.081728,-0.168514,...,0.138165,0.684888,-0.235546,-0.237277,0.234673,0.616999,0.226965,-0.017151,-0.196661,0.199716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248851,1181,79,-0.220790,-0.109746,-0.078201,0.014953,-0.422946,0.059752,0.127205,-0.027481,...,0.115354,0.573477,-0.167752,-0.340064,0.217530,0.403942,0.102886,-0.051067,0.019311,0.290759
248852,1181,80,-0.208558,-0.074538,-0.129259,0.055462,-0.374811,0.140472,0.060977,-0.049488,...,0.080577,0.511815,-0.079852,-0.271697,0.289177,0.311276,0.059422,-0.062565,0.091055,0.204489
248853,1181,81,-0.268144,-0.092291,-0.170665,0.041967,-0.388190,0.156224,0.056734,0.026896,...,0.073975,0.557348,-0.140225,-0.310226,0.306960,0.315961,0.043454,-0.085057,0.108163,0.259592
248854,1181,82,-0.110338,-0.162252,-0.162132,-0.055742,-0.255060,0.098632,-0.000473,0.002854,...,0.080956,0.505160,-0.112232,-0.156384,0.108615,0.345401,0.023158,0.068246,0.065752,0.238986


In [55]:
# group the rows by speech number and compute the mean of each group
df_speech = df.groupby("speech_number").mean()

# reset the index of the dataframe and rename the columns
df_speech = df_speech.reset_index().rename(columns={"index": "speech_number"})

# re-order the columns of the dataframe
df_speech = df_speech[["speech_number"] + ["dim_" + str(i) for i in range(300)]]


In [58]:
df_speech['speech_number'] = df_speech['speech_number'] +1

In [60]:
df_speech

,speech_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,1,-0.170942,-0.105910,-0.129880,0.044871,-0.433940,0.118041,0.062991,-0.067675,0.274816,...,0.109752,0.575568,-0.118102,-0.293912,0.289865,0.379960,0.089991,-0.062723,0.049281,0.202227
1,2,-0.173813,-0.103157,-0.128619,0.048244,-0.440676,0.122350,0.066660,-0.079281,0.270524,...,0.111625,0.580856,-0.113213,-0.295069,0.299558,0.394135,0.086616,-0.072185,0.041569,0.202509
2,3,-0.194960,-0.112100,-0.144768,0.067966,-0.438016,0.142917,0.058852,-0.058497,0.297790,...,0.116083,0.572594,-0.131115,-0.305181,0.293579,0.350674,0.079339,-0.048725,0.080621,0.205579
3,4,-0.194948,-0.093101,-0.137795,0.059659,-0.426905,0.140299,0.064522,-0.063876,0.262760,...,0.099754,0.556804,-0.106051,-0.296029,0.310600,0.356272,0.076125,-0.077547,0.066590,0.198399
4,5,-0.207889,-0.086608,-0.142335,0.058436,-0.409174,0.143714,0.065472,-0.041583,0.275479,...,0.091586,0.553238,-0.098212,-0.293806,0.304865,0.333311,0.064703,-0.064046,0.085690,0.208489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,1178,-0.191878,-0.105878,-0.142410,0.076949,-0.440596,0.155736,0.064276,-0.082755,0.288009,...,0.110560,0.562506,-0.114370,-0.302296,0.316580,0.354971,0.081491,-0.068461,0.064869,0.181488
1177,1179,-0.183174,-0.101122,-0.115999,0.044669,-0.427698,0.110772,0.086859,-0.061124,0.287834,...,0.107990,0.563974,-0.117099,-0.304133,0.281405,0.368485,0.093947,-0.054414,0.051613,0.210667
1178,1180,-0.230263,-0.096615,-0.150812,0.087899,-0.433378,0.175474,0.069312,-0.053042,0.310024,...,0.100679,0.558198,-0.111992,-0.318232,0.333079,0.311824,0.070564,-0.060856,0.107035,0.197230
1179,1181,-0.198900,-0.093162,-0.115447,0.072432,-0.435185,0.129623,0.095446,-0.068955,0.307528,...,0.112968,0.553645,-0.108793,-0.316181,0.300278,0.344667,0.090590,-0.058631,0.065423,0.194988


In [66]:
df_data.head(20)

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,['governor andhra pradesh shri e l narasimhan ...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,['chief minister gujarat shri vijay rupani ji ...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,['delighted gift city inaugurate india first i...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,['congratulate finance minister arun jaitley j...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,['sri pejavar math respected sri vishwesh tirt...,sri pejavar math respected sri vishwesh tirth ...
5,NMS6,2017-02-21,Namaskaram. Greetings to everyone. Swami Nirvi...,Text of PM’s inaugural address (via video conf...,"['namaskaram', 'greeting everyone', 'swami nir...",namaskaram greeting everyone swami nirvinanand...
6,NMS7,2017-02-26,"My dear countrymen, Namaskar. Winter is on its...",English Translation of the text of PM’s ‘Mann ...,"['dear countryman namaskar', 'winter way', 'va...",dear countryman namaskar winter way vasant sea...
7,NMS8,2017-03-02,"Swami Chidanand Saraswati ji,Shankaracharya Di...",Text of PM’s inaugural address (via video conf...,['swami chidanand saraswati ji shankaracharya ...,swami chidanand saraswati ji shankaracharya di...
8,NMS9,2017-03-07,"Yesterday I was near mother Ganges, today I’m ...",English rendering of the text of PMs speech on...,['yesterday near mother ganges today mother na...,yesterday near mother ganges today mother narm...
9,NMS10,2017-03-07,"Chief Minister of Gujarat, Shri Vijay Rupani \...",English rendering of the text of PM's speech a...,['chief minister gujarat shri vijay rupani col...,chief minister gujarat shri vijay rupani colle...


In [61]:
df_data.drop(18, inplace=True)

In [63]:
df_data = df_data.reset_index(drop=True)

In [67]:
df_speech.insert(0, "index", df_data["index"])
df_speech.insert(1, "Date", df_data["Date"])

In [71]:
df_speech

,index,Date,speech_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,NMS1,2017-01-03,1,-0.170942,-0.105910,-0.129880,0.044871,-0.433940,0.118041,0.062991,...,0.109752,0.575568,-0.118102,-0.293912,0.289865,0.379960,0.089991,-0.062723,0.049281,0.202227
1,NMS2,2017-01-09,2,-0.173813,-0.103157,-0.128619,0.048244,-0.440676,0.122350,0.066660,...,0.111625,0.580856,-0.113213,-0.295069,0.299558,0.394135,0.086616,-0.072185,0.041569,0.202509
2,NMS3,2017-01-09,3,-0.194960,-0.112100,-0.144768,0.067966,-0.438016,0.142917,0.058852,...,0.116083,0.572594,-0.131115,-0.305181,0.293579,0.350674,0.079339,-0.048725,0.080621,0.205579
3,NMS4,2017-02-01,4,-0.194948,-0.093101,-0.137795,0.059659,-0.426905,0.140299,0.064522,...,0.099754,0.556804,-0.106051,-0.296029,0.310600,0.356272,0.076125,-0.077547,0.066590,0.198399
4,NMS5,2017-02-05,5,-0.207889,-0.086608,-0.142335,0.058436,-0.409174,0.143714,0.065472,...,0.091586,0.553238,-0.098212,-0.293806,0.304865,0.333311,0.064703,-0.064046,0.085690,0.208489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,NMS1178,2023-04-09,1178,-0.191878,-0.105878,-0.142410,0.076949,-0.440596,0.155736,0.064276,...,0.110560,0.562506,-0.114370,-0.302296,0.316580,0.354971,0.081491,-0.068461,0.064869,0.181488
1177,NMS1179,2023-04-12,1179,-0.183174,-0.101122,-0.115999,0.044669,-0.427698,0.110772,0.086859,...,0.107990,0.563974,-0.117099,-0.304133,0.281405,0.368485,0.093947,-0.054414,0.051613,0.210667
1178,NMS1180,2023-04-12,1180,-0.230263,-0.096615,-0.150812,0.087899,-0.433378,0.175474,0.069312,...,0.100679,0.558198,-0.111992,-0.318232,0.333079,0.311824,0.070564,-0.060856,0.107035,0.197230
1179,NMS1181,2023-04-14,1181,-0.198900,-0.093162,-0.115447,0.072432,-0.435185,0.129623,0.095446,...,0.112968,0.553645,-0.108793,-0.316181,0.300278,0.344667,0.090590,-0.058631,0.065423,0.194988


In [72]:
word2vec_NM = open("word2vec_NM.pickle", "wb")
pickle.dump(df_speech, word2vec_NM)
word2vec_NM.close()